In [1]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet "tensorflow-hub>=0.7.0"
!pip3 install --quiet seaborn

In [6]:
from absl import logging

import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import json
import pandas as pd
from tqdm import tqdm

In [3]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [4]:
tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
stocknet_dataset_filepath = './stocknet-dataset-master'

In [8]:
preprocessed_prices_filepath = stocknet_dataset_filepath + '/price/preprocessed'
preprocessed_tweets_filepath = stocknet_dataset_filepath + '/tweet/preprocessed'

company_to_price_df = {}
company_to_tweets = {}

for filename in os.listdir(preprocessed_prices_filepath):
    with open(preprocessed_prices_filepath + '/' + filename) as file:
        company_name = filename.split('.')[0]
        
        # Not enough data for GMRE
        if company_name == 'GMRE':
            continue
        df = pd.read_csv(file, sep='\t')
        df.columns = ['date', 'open', 'high', 'low', 'close', 'adjust_close', 'volume']
        company_to_price_df[company_name] = df

for filename in tqdm(os.listdir(preprocessed_tweets_filepath)):
    company_name = filename.split('.')[0]
    dates_to_tweets = {}
    for tweet_filename in os.listdir(preprocessed_tweets_filepath + '/' + filename):
        with open(preprocessed_tweets_filepath + '/' + filename + '/' + tweet_filename) as file:
            list_of_tweets = []
            for line in file:
                tweet_json = json.loads(line)
                list_of_tweets.append(tweet_json)
            dates_to_tweets[tweet_filename] = list_of_tweets
    company_to_tweets[company_name] = dates_to_tweets

100%|██████████| 87/87 [00:01<00:00, 61.28it/s]


In [18]:
#print(company_to_tweets.keys())
#print(dates_to_tweets.keys())
print(company_to_tweets['AAPL']['2015-10-02'][0])

{'text': ['rt', '$', 'tsla', 'hft', 'algos', 'triggered', 'buy', 'in', 'sigma-x', ',', 'crossfinder', ',', 'ats', ',', 'lx', '@', '08:28', ',', 'p', '/', 't', '245.00', 'quant', '$', 'msft', '$', 'fb', '$', 'gpro', '$', 'amzn', '$', 'goog', '$', 'aapl', '$', 'nflx', '$', 'qqq'], 'created_at': 'Fri Oct 02 12:29:15 +0000 2015', 'user_id_str': '242469235'}


In [ ]:
# company_to_tweets maps each company name to a dictionary of dates and list of tweets. Schema:

{
    company_name: 
    {
        date: [list of tweets + metadata]
        ...
    }
    ...
}

In [ ]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Reduce logging output.
logging.set_verbosity(logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))
  
  for company in tqdm(company_to_tweets.keys()):
    for date in company_to_tweets[company].keys():
      messages = []
      for j in range(len(company_to_tweets[company][date])):
        messages.append(' '.join(company_to_tweets[company][date][j]['text']))
      message_embeddings = session.run(embed(messages))
      print(message_embeddings)
      for k in range(len(company_to_tweets[company][date])):
        company_to_tweets[company][date][k]['embedding'] = list(message_embeddings[k])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
